In [131]:
from bs4 import BeautifulSoup
import requests
import warnings
import time
import random
from functools import reduce
from z3 import *

In [86]:
warnings.filterwarnings('ignore')

In [63]:
def get_x_y(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')
    page = soup.find('p').getText()
    token = page.split(' ')
    x = int(token[3])
    y = int(token[-1][0:-1])
    return x,y

In [73]:
def check_if_at_start(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')
    page = soup.find('p').getText()
    if 'You are coming back home from a hard day sieving numbers at the river.' in page:
        return True
    else:
        return False 

In [81]:
def pick(num):
    y = random.randint(0,1)
    return num[y]

In [145]:
def get_two_guesses(session):
    c1 = 0
    c2 = 0

    p = session.get('https://giant-goannas.ctfcompetition.com/', verify=False)

    p2 = session.get('https://giant-goannas.ctfcompetition.com/start', verify=False)

    p3 = session.get('https://giant-goannas.ctfcompetition.com/lake', verify=False)

    x1,y1 = get_x_y(p3.text)
    c1 = pick([x1,y1])
    
    p4 = session.post('https://giant-goannas.ctfcompetition.com/lake',  {'number':c1}, verify=False)


    if check_if_at_start(p4.text):
        print "start again"
        c1 = 0
    else:
        print "good first choice"
        x2,y2 = get_x_y(p4.text)
        c2 = pick([x2,y2])
        p5 = session.post('https://giant-goannas.ctfcompetition.com/lake',  {'number':c2}, verify=False)

        if check_if_at_start(p5.text):
            print "start again"
            c2 = 0
        else:
            print "good second choice"
            print c2
            
    if c1 >0 and c2 > 0:
        return c1,c2, get_x_y(p5.text)
    else:
        return None, None, None

In [146]:
def marshall_initial(session):
    c1, c2, X_Y = get_two_guesses(session)
    while c1 == None:
        c1, c2, X_Y = get_two_guesses(session)
    return c1,c2, X_Y

In [147]:
def question_response(session,prng):
    data = []
    for i,C in enumerate(F[2:]):
        p6 = session.post('https://giant-goannas.ctfcompetition.com/lake',  {'number':C}, verify=False)
        cookie_ = requests.utils.dict_from_cookiejar(session.cookies)
        if "CTF" in p6.text:
            print p6.text
            return p6.text 
        data.append((cookie_, p6.text, i, C))
    return data

In [148]:
session = requests.session()
c1,c2, X_Y =  marshall_initial(session)

first_correct_number = c1
second_correct_number = c2

third_question_options = X_Y

possibilites = my_solver(first_correct_number,second_correct_number,SIZE=36)
x,y = final_check(third_question_options,possibilites)
F = generate_random(x,y,100000)

start again
start again
good first choice
good second choice
1889866598
(1549775032, 3696378569)
(122255545, 2785632374)


In [149]:
question_response(session,F)

Congratulations, here is your reward: CTF{_!_aRe_y0U_tH3_NSA_:-?_!_}


u'Congratulations, here is your reward: CTF{_!_aRe_y0U_tH3_NSA_:-?_!_}'

In [99]:
def my_solver(g_1,g_2,SIZE=36):
    x = BitVec('x', SIZE)
    y = BitVec('y', SIZE)
    P1 = (2*x + 3) % 4646704883
    P2 = (3*y + 9) % 4646704883

    x2 = BitVec('x2', SIZE)
    y2 = BitVec('y2', SIZE)
    P1_2 = (2*P1 + 3) % 4646704883
    P2_2 = (3*P2 + 9) % 4646704883

    s = Solver()
    s.add(P1 < 4646704883)
    s.add(P2 < 4646704883)
    s.add(x < 4646704883)
    s.add(y < 4646704883)
    s.add(P1^P2 == g_1)


    s.add(P1_2 < 4646704883)
    s.add(P2_2 < 4646704883)
    s.add(x2 < 4646704883)
    s.add(y2 < 4646704883)
    s.add(P1_2^P2_2 == g_2)

    keep = []
    while s.check() == sat:
        x_p = s.model()[x].as_long()
        y_p = s.model()[y].as_long()
        X = (2* x_p + 3) % 4646704883
        Y = (3* y_p + 9) % 4646704883
        K = X ^ Y
        s.add(Or(x != s.model()[x], y != s.model()[y]))
        if K == g_1:
            print(x_p, y_p)
            keep.append((x_p,y_p))
    return keep

In [100]:
def generate_random(x_i,y_i,how_many):
    K = [] 
    a = (2 * x_i + 3) % 4646704883
    b = (3 * y_i + 9) % 4646704883
    K.append(a^b)
    for i in range(0,how_many):
        a = (2 * a + 3) % 4646704883
        b = (3 * b + 9) % 4646704883
        K.append(a^b)
    return K

In [101]:
def get_correct_initial(possibilites, test):
    for i,p in enumerate(possibilites):
        if test in generate_random(p[0],p[1],5):
            return i
    return None 

In [102]:
def final_check(guesses,possibilites):
    for i,g in enumerate(guesses):
        i = get_correct_initial(possibilites,g)
        if i != None:
            x,y = possibilites[i]
            return(x,y)
    return None